In [1]:
import numpy as np
import torch
from torch.distributions.multivariate_normal import MultivariateNormal

import matplotlib as plt
import matplotlib.image as mpimg

import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from skimage import io
import os

import math
import requests

from queue import PriorityQueue

In [2]:
plotly.__version__
pio.renderers.default = "iframe"

In [3]:
np.__version__

'1.16.6'

In [4]:
np.random.seed(0)

In [5]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

In [6]:
%load_ext autoreload
%autoreload 2

from model import Bicycle, Map
from mppi import MPPI, MPPI_path_follower
from path_planner import a_star_planner
from sim import Simulation


In [7]:
torch.cuda.empty_cache() 

In [8]:
def get_expert_controls(control_history, horizon_depth, ctrl_dim, interval_start = 0, interval_end = None, dev="cpu"):
    if(interval_end is None):
        interval_end = len(control_history)-horizon_depth
    expert_controls = torch.empty([interval_end-interval_start, horizon_depth, ctrl_dim], dtype = torch.float, device = dev)
    for i in range(interval_start, interval_end):
        expert_controls[i-interval_start,:,:] = torch.stack(control_history[i:i+horizon_depth])        
    return expert_controls

In [9]:
avoid = []

rect = [(20, 0, 80, 50),(20,51,80, 100) ]
circle= []

state_dim = 4
ctrl_dim = 2
u_min = (0., -3.)
u_max = (10., 3.)
step = .1
follow_depth = 16 # note: if these parameters are changed, the mppi ones should be changed to match
horizon_depth = 32

In [ ]:
expert_controls = torch.tensor([],device = torch.device('cuda:0')) # used to reset expert_controls from constantly appending
a_star_sim = None
#
for i in [1, 3, 5]:
    for j in range(45, 56, 2):
        a_star_robot = Bicycle(i, j, 0, 0)
        goal = (90, 50, 3 , 100)

        mppi_sample_center= torch.tensor([0., 0.])
        mppi_sample_covariance = torch.tensor([[16.,0.],[0., 1.]])

        env = Map(goal_point = goal, avoidance_points=avoid, rect_obstacles=rect,circle_obstacles=circle, device = torch.device('cuda:0'))
        a_star_map = Map(goal_point = (a_star_robot.x, a_star_robot.y,2,2), avoidance_points=avoid, rect_obstacles=rect,circle_obstacles=circle, device = torch.device('cuda:0'))

        def a_star_heuristic(state):     
            return env.terminal_state_cost_batch(state)

        a_star_plan = a_star_planner(a_star_robot, state_dim, ctrl_dim, u_min, u_max, 
                          env.get_obstacles_batch, env.get_distance_batch,
                         iterations = 50000, goal_point = env.goal_point, angle_density = 32, grid_interval = 0.2, goal_tolerance =0.1, 
                                           timestep = step, device=torch.device('cuda:0'))
        
        print ("plan ready")
        a_star_follower = MPPI_path_follower(a_star_robot, state_dim, ctrl_dim, 
                          mppi_sample_center, mppi_sample_covariance,  u_min, u_max, 
                          a_star_robot.dynamics_batch_horizon, a_star_map, 
                                             a_star_map.running_cost_batch_horizon, a_star_map.terminal_state_cost_batch, a_star_plan.path, 
                         num_samples = 4096, horizon = follow_depth, lambda_=.1, sample_null_action = True,
                          timestep = step, device=torch.device('cuda:0'))
        print("demo generated")
        
        a_star_sim = Simulation(a_star_robot, a_star_follower, a_star_map, goal_tolerance = 1, timestep = step)
        
        a_star_sim.run(100, True, True, True, 10, 1)

        new_controls = get_expert_controls(a_star_robot.control_history, horizon_depth, ctrl_dim, dev = torch.device('cuda:0'))
        expert_controls = torch.cat((expert_controls, new_controls), 0)
        new_controls[:,:,1] = -new_controls[:,:,1] # negate steering controls
        expert_controls = torch.cat((expert_controls, new_controls), 0)

        print(i, j, expert_controls.shape)

#ideally we wouldn't need to pass in goal_point and could just run off of cost functions


plan ready
demo generated
(1, 45, torch.Size([132, 32, 2]))
plan ready
demo generated
(1, 47, torch.Size([250, 32, 2]))
plan ready
demo generated
(1, 49, torch.Size([370, 32, 2]))
plan ready
demo generated
(1, 51, torch.Size([496, 32, 2]))
plan ready
demo generated
(1, 53, torch.Size([632, 32, 2]))
plan ready
demo generated
(1, 55, torch.Size([754, 32, 2]))
plan ready
demo generated
(3, 45, torch.Size([870, 32, 2]))
plan ready
demo generated
(3, 47, torch.Size([986, 32, 2]))
plan ready
demo generated
(3, 49, torch.Size([1100, 32, 2]))
plan ready
demo generated
(3, 51, torch.Size([1212, 32, 2]))
plan ready
demo generated
(3, 53, torch.Size([1328, 32, 2]))
plan ready
demo generated
(3, 55, torch.Size([1446, 32, 2]))
plan ready
demo generated
(5, 45, torch.Size([1564, 32, 2]))
plan ready
demo generated
(5, 47, torch.Size([1676, 32, 2]))
plan ready
demo generated
(5, 49, torch.Size([1784, 32, 2]))
plan ready
demo generated
(5, 51, torch.Size([1892, 32, 2]))
plan ready
demo generated
(5, 53

In [ ]:
a_star_sim.draw_explored(a_star_plan.path, True)

In [ ]:
a_star_sim.display_run()
a_star_sim.display_history()

In [ ]:
mppi_robot = Bicycle(5, 46, 0, 0)
mppi_goal = (90, 50, 2 , 2)

mppi_sample_center= torch.tensor([0., 0.])
mppi_sample_covariance = torch.tensor([[16.,0.],[0., .1]])
# note: if these parameters are changed, the a-star ones should be changed to match

mppi_map = Map(goal_point = mppi_goal, avoidance_points=avoid, rect_obstacles=rect,circle_obstacles=circle, device = torch.device('cuda:0'))
mppi_controller = MPPI(mppi_robot, state_dim, ctrl_dim, 
                  mppi_sample_center, mppi_sample_covariance,  u_min, u_max, 
                  mppi_robot.dynamics_batch_horizon, mppi_map.running_cost_batch_horizon, mppi_map.terminal_state_cost_batch,expert_rollouts = expert_controls,
                 num_samples = 1982, horizon = horizon_depth, lambda_=.1, sample_null_action = True,
                  timestep = step, device=torch.device('cuda:0'))

mppi_sim = Simulation(mppi_robot, mppi_controller, mppi_map, timestep = step)


In [ ]:
mppi_sim.run(iterations=150, write_snapshots = True, 
      write_rollouts=True, write_controls = True, write_rollouts_num = 10,
     write_rate = 1)
mppi_sim.display_run()
mppi_sim.display_history()